# Review of Possible Changed Data Structure
11/29/23<br>
after running successfully on Monday & Tuesday, I’m getting the following error in python3 scripts/api-request.py:

KeyError: "['vehicle_docks_available'] not found in axis"
<br>

# import libraries

In [1]:
import pandas as pd
import requests
import datetime
import pytz

# review vehicle types

In [2]:
url = f"https://gbfs.lyft.com/gbfs/2.3/chi/en/vehicle_types.json?"

#run the query
response = requests.get(url)
data = response.json()
response

<Response [200]>

In [3]:
#navigate through dictionaries to find station data
#data.keys()
data['data'].keys()

dict_keys(['vehicle_types'])

In [4]:
central_tz = pytz.timezone('America/Chicago')
status_timestamp = datetime.datetime.utcfromtimestamp(data['last_updated']).replace(tzinfo=pytz.utc)
status_timestamp = status_timestamp.astimezone(central_tz).strftime('_%Y_%m_%d_%I%M%p')
status_timestamp

'_2023_11_29_0937AM'

In [5]:
#look at all data
data['data']['vehicle_types']

[{'vehicle_type_id': '1',
  'form_factor': 'bicycle',
  'propulsion_type': 'human'},
 {'vehicle_type_id': '2',
  'max_range_meters': 54717.56,
  'form_factor': 'bicycle',
  'propulsion_type': 'electric_assist'},
 {'vehicle_type_id': '3',
  'max_range_meters': 70810.95999999999,
  'form_factor': 'scooter',
  'propulsion_type': 'electric'}]

# review station info

In [6]:
url = f"https://gbfs.lyft.com/gbfs/2.3/chi/en/station_information.json?"

#request data
response = requests.get(url)
data = response.json()
response

<Response [200]>

In [7]:
#navigate through dictionaries to find station data
#data.keys()
data['data'].keys()

dict_keys(['stations'])

In [8]:
#look at the first station
data['data']['stations'][0]

{'lat': 41.916907,
 'name': 'Parkside Ave & Armitage Ave',
 'rental_uris': {'android': 'https://chi.lft.to/lastmile_qr_scan',
  'ios': 'https://chi.lft.to/lastmile_qr_scan'},
 'lon': -87.767173,
 'short_name': '21354',
 'station_id': '1871993715238611622',
 'capacity': 15}

In [8]:
df_station_info = pd.DataFrame(data['data']['stations'], columns=['station_id','name','lat','lon','capacity','short_name'])
df_station_info.head()

station_id                    name        lat  \
0  a3af2737-a135-11e9-9cda-0a87ae2ba916  Clark St & Elmdale Ave  41.990860   
1  a3b20960-a135-11e9-9cda-0a87ae2ba916     Damen Ave & 51st St  41.800908   
2  a3a9c0f6-a135-11e9-9cda-0a87ae2ba916     Morgan St & 31st St  41.837800   
3  94788869-1849-43f6-99e1-2eacbc3fab03    Morgan St & Lake St*  41.885492   
4  a3aae35a-a135-11e9-9cda-0a87ae2ba916   Calumet Ave & 35th St  41.831379   

         lon  capacity    short_name  
0 -87.669724        15  KA1504000148  
1 -87.674684         7           554  
2 -87.651140        11  TA1308000046  
3 -87.652289        20  chargingstx4  
4 -87.618034        15         13345

# review station status

### review data structures

In [9]:
#run the query
url = f"https://gbfs.lyft.com/gbfs/2.3/chi/en/station_status.json?"
response = requests.get(url)
data = response.json()
response

<Response [200]>

In [10]:
#navigate through dictionaries to find station data
# data.keys()
data['data'].keys()

dict_keys(['stations'])

In [12]:
#look at the first station
data_station_status = data['data']['stations']
data_station_status[0]

{'num_bikes_disabled': 0,
 'num_bikes_available': 0,
 'is_renting': 1,
 'num_docks_available': 15,
 'last_reported': 1701272230,
 'is_installed': 1,
 'is_returning': 1,
 'num_ebikes_available': 0,
 'vehicle_types_available': [{'count': 0, 'vehicle_type_id': '1'},
  {'count': 0, 'vehicle_type_id': '2'}],
 'num_docks_disabled': 0,
 'station_id': 'a3af2737-a135-11e9-9cda-0a87ae2ba916'}

In [13]:
#read into dataframe and rename columns
df_station_status= pd.DataFrame(data_station_status, columns=
        ['station_id','num_bikes_available','vehicle_docks_available',
         'vehicle_types_available','num_bikes_disabled','num_docks_available','num_ebikes_available'])

In [16]:
df_station_status.head()

station_id  num_bikes_available  \
0  a3af2737-a135-11e9-9cda-0a87ae2ba916                    0   
1  a3b20960-a135-11e9-9cda-0a87ae2ba916                    4   
2  a3a9c0f6-a135-11e9-9cda-0a87ae2ba916                    0   
3  94788869-1849-43f6-99e1-2eacbc3fab03                   14   
4  a3aae35a-a135-11e9-9cda-0a87ae2ba916                   12   

   vehicle_docks_available                            vehicle_types_available  \
0                      NaN  [{'count': 0, 'vehicle_type_id': '1'}, {'count...   
1                      NaN  [{'count': 0, 'vehicle_type_id': '1'}, {'count...   
2                      NaN  [{'count': 0, 'vehicle_type_id': '1'}, {'count...   
3                      NaN  [{'count': 10, 'vehicle_type_id': '1'}, {'coun...   
4                      NaN  [{'count': 10, 'vehicle_type_id': '1'}, {'coun...   

   num_bikes_disabled  num_docks_available  num_ebikes_available  n_classic  \
0                   0                   15                     0          0   
1                   0                    3                     4          0   
2                   1                   10                     0          0   
3                   0                    4                     4         10   
4                   0                    3                     2         10   

   n_electric  n_scooters  
0           0           0  
1           4           0  
2           0           0  
3           4           0  
4           2           0

### review vehicle type dictionary

In [14]:
#extract vehicle type info
df_station_status['n_classic'] = df_station_status['vehicle_types_available'].apply(
    lambda x: next((item['count'] for item in x if item['vehicle_type_id'] == '1'), 0))
df_station_status['n_electric'] = df_station_status['vehicle_types_available'].apply(
    lambda x: next((item['count'] for item in x if item['vehicle_type_id'] == '2'), 0))
df_station_status['n_scooters'] = df_station_status['vehicle_types_available'].apply(
    lambda x: next((item['count'] for item in x if item['vehicle_type_id'] == '3'), 0))

df_station_status[['station_id','n_classic','n_electric','n_scooters']].head()

station_id  n_classic  n_electric  n_scooters
0  a3af2737-a135-11e9-9cda-0a87ae2ba916          0           0           0
1  a3b20960-a135-11e9-9cda-0a87ae2ba916          0           4           0
2  a3a9c0f6-a135-11e9-9cda-0a87ae2ba916          0           0           0
3  94788869-1849-43f6-99e1-2eacbc3fab03         10           4           0
4  a3aae35a-a135-11e9-9cda-0a87ae2ba916         10           2           0

### review dock types dictionary

In [17]:
df_station_status['vehicle_docks_available']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1665   NaN
1666   NaN
1667   NaN
1668   NaN
1669   NaN
Name: vehicle_docks_available, Length: 1670, dtype: float64

In [18]:
df_station_status['dock_type_ids'] = df_station_status[
    'vehicle_docks_available'].apply(lambda x: x[0]['vehicle_type_ids'] if x else None)
df_station_status['n_docks'] = df_station_status['vehicle_docks_available'].apply(lambda x: x[0]['count'] if x else None)

TypeError: 'float' object is not subscriptable